In [1]:
import pandas as pd
import numpy as np
import os

min_dataV = 0
max_dataV = [0,0,0]

#19년 20년 월별 데이터 생성
data2019 = {}
data2020 = {}

def csv_to_dataFrame(year,dataF):
    global min_dataV,max_dataV
    year = str(year)
    
    for indx,csvF in enumerate(os.listdir('./seoul_subway_data/{}'.format(year))):
        #19년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
        if not 'data' + csvF[-10:-4] in dataF.keys():
            dataF.setdefault('data' + csvF[-10:-4])
        else:
            continue

        try:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'UTF-8',index_col=False)
        except Exception as e:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'EUC-KR',index_col=False)
        dataF['data' + csvF[-10:-4]] = data

    #2019년 데이터에서 불필요한 데이터 조정
    for dataY in dataF.keys():
        if '역ID' in dataF[dataY].columns:
            dataF[dataY].drop('등록일자',axis=1,inplace = True)
            dataF[dataY].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
            dataF[dataY].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
            dataF[dataY].rename(columns = {'역명' : '승차총승객수'},inplace = True)
            dataF[dataY].rename(columns = {'역ID' : '역명'},inplace = True)
            
        dataF[dataY]['유동인구수'] = dataF[dataY]['하차총승객수'] - dataF[dataY]['승차총승객수']
        
        if max_dataV[0] < dataF[dataY]['승차총승객수'].max():
            max_dataV[0] = dataF[dataY]['승차총승객수'].max()
        
        if max_dataV[1] < dataF[dataY]['하차총승객수'].max():
            max_dataV[1] = dataF[dataY]['하차총승객수'].max()
            
        if max_dataV[2] < dataF[dataY]['유동인구수'].max():
            max_dataV[2] = dataF[dataY]['유동인구수'].max()            

csv_to_dataFrame(2019,data2019)
csv_to_dataFrame(2020,data2020)




<h1>데이터 정리 최종</h1>

In [2]:
from selenium import webdriver
import time
import os, csv

option = webdriver.ChromeOptions()
option.add_argument('headless')

# os.path.abspath('chromedriver')

driver = webdriver.Chrome(os.path.abspath('chromedriver'),options=option)
driver.get('https://gits.gg.go.kr/gtdb/web/trafficDb/railRoad/TransitSWPass.do')

time.sleep(3)

def setRadio():
    driver.find_element_by_xpath('//*[@id="radio1"]').click()

def setSelect1(sel1):
    driver.find_element_by_xpath("//select[@name='select1']/option[text()='%s']" %sel1).click()
    
def setSelect2(sel2):
    driver.find_element_by_xpath("//select[@name='select2']/option[text()='%s']" %sel2).click()

#soup로 고정된 html 값에서 radio1 버튼 찾아와서 클릭하기
setRadio()

#rad 버튼 클릭후 동적으로 변경된 웹페이지에서 select1아이디 값 가져오기
select1 = driver.find_element_by_id('select1')
t_sel1 = select1.text.split('\n')
sel1 = []
for i,t in enumerate(t_sel1):
    if not t.strip() == '':
        sel1.append(t.strip())

#시 - 역명 넣을 csv파일 생상(기존에 있는건 삭제)
if os.path.exists('./seoul_subway_data/구_역명.csv'):
    os.remove('./seoul_subway_data/구_역명.csv')

with open('./seoul_subway_data/구_역명.csv','w') as csvFile:
    writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])    
    writer.writeheader()
    
#sel1 값을 넣은 후 sel2값 넣고 검색 후 list값들을 가져온 다음 데이터 정리
for i in sel1:
    setSelect1(i)
    
    sel2 = driver.find_element_by_id('select2')
    t_sel2 = sel2.text.split('\n')
    for a in range(len(t_sel2)):
        t_sel2[a] = t_sel2[a].strip()
        #전체 선택 혹은 비어있는 경우는 넘어가기 (행정 구역만 필요하니까)
        if '전체선택' == t_sel2[a] or '' == t_sel2[a]:
            continue
        
        setSelect2(t_sel2[a])
        driver.find_element_by_xpath('//*[@id = "search"]').click()
        li_loc = (driver.find_element_by_id('selList')).text.split('\n')

        for i in li_loc:
            #가져온 list값은 매봉(1호선) 이런식으로 돼있으므로 ()를 기준으로 역이름과 호선명 나눠주기
            st_idx = i.find('(') + 1
            f_idx = i.find(')')
            t_sel2[a] = t_sel2[a].replace('_','')
            #csv파일에 바로 넣기
            with open('./seoul_subway_data/구_역명.csv','a') as csvFile:
                writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])
                writer.writerow({'구':t_sel2[a],'역명':i[:st_idx-1]})

gu_sta_name = pd.read_csv('./seoul_subway_data/구_역명.csv')

driver.quit()

def insert_gu(dataF):
    global min_dataV,max_dataV
    newData_min = {'사용일자':'', '노선명':'', '역명':'', '승차총승객수':min_dataV, '하차총승객수':min_dataV, '등록일자':'', '유동인구수':min_dataV}
    newData_max = {'사용일자':'', '노선명':'', '역명':'', '승차총승객수':max_dataV[0], '하차총승객수':max_dataV[1], '등록일자':'', '유동인구수':max_dataV[2]}
    #데이터에 구 추가
    for dataY in dataF.keys():
        if not '구' in dataF[dataY].columns:    
            dataF[dataY] = pd.merge(dataF[dataY],gu_sta_name,on='역명',how = 'inner')

            if True in dataF[dataY].duplicated():
                dataF[dataY] = dataF[dataY].drop_duplicates()
                dataF[dataY].reset_index(inplace=True)
            if 'index' in dataF[dataY].columns:
                dataF[dataY].drop(columns='index',inplace=True)
                
        dataF[dataY] = dataF[dataY].append(newData_min,ignore_index=True)
        dataF[dataY] = dataF[dataY].append(newData_max,ignore_index=True)
    
insert_gu(data2019)
insert_gu(data2020)

DATA = {}
if not 'data2019' in DATA.keys():
    DATA['data2019'] = data2019
if not 'data2020' in DATA.keys():
    DATA['data2020'] = data2020

<h1>그냥 바로 해버리면 전국에 칠해지니까 서울지역으로 json 범위 축소 </h1><br><p>근데 왜 한번에 안되고 여러번에 거쳐야 되는지는 모르겠음...</p>

In [43]:
import json
import folium
import webbrowser

geo_path = './map/skorea-municipalities-2018-geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

#json 데이터 안다듬어주면 전국단위로 난리남
print('json 처리 전 지역 수 : ',len(geo_str['features']))
min_len_geoStr = False
k = [-1,-2]
#서울 지역 코드 값 = 1로 시작
#왜인지는 모르겠는데 한번에 처리가 안되서 이전 카운트 개수하고 현재 카운트 개수에 차이가 없을 때 break
while min_len_geoStr == False:
    if k[0] == k[1]:
        min_len_geoStr = True
    k[0] = len(geo_str['features'])
    for i,t in enumerate(geo_str['features']):
        if not t['properties']['code'][0] == '1':
            del geo_str['features'][i]
    k[1] = len(geo_str['features'])        

print('json 처리 후 지역 수 : ',len(geo_str['features']))

class mapping:
    global geo_str,DATA
    
    def __init__(self,dataF,ym,mt):
        self.map=[folium.Map(location=[37.5502, 126.982], zoom_start=11, min_zoom=11,
                    tiles='stamentoner') for i in range(3)]
        
        self.dataF = dataF
        
        self.ym = ym
        self.mt = mt
        self.drawMap()


    def drawMap(self):
        #맵 그려주기
        if self.mt == '1':
            folium.Choropleth(
                geo_data=geo_str,
                data=self.dataF['data' + self.ym], 
                columns=['구','승차총승객수'], 
                key_on='feature.properties.name',
                fill_color='PuRd',
                legend_name='지하철 승차인구 수'
            ).add_to(self.map[0])
            self.saveMap()

        elif self.mt == '2':
            folium.Choropleth(
                geo_data=geo_str,
                data=self.dataF['data' + self.ym], 
                columns=['구','하차총승객수'], 
                key_on='feature.properties.name',
                fill_color='BuPu',
                legend_name='지하철 하차인구 수'
            ).add_to(self.map[1])
            self.saveMap()

        elif self.mt == '3':
            folium.Choropleth(
                geo_data=geo_str,
                data=self.dataF['data' + self.ym], 
                columns=['구','유동인구수'], 
                key_on='feature.properties.name',
                fill_color='OrRd',
                legend_name='지하철 유동인구 수'
            ).add_to(self.map[2])
            self.saveMap()                
    
    #그린 맵 파일 확인 후 저장
    def saveMap(self):
        #mt == maptype
        #쥬피터에서 보여주면 한글 깨져서 동일 경로 map 디렉토리에 해당 맵을 만들어줌
        
        #mt == 0 승차
        if self.mt == '1':
            if os.path.exists('./map/getOn'):
                if os.path.exists('./map/getOn/{}.html'.format(self.ym)):
                    os.remove('./map/getOn/{}.html'.format(self.ym))
            else:
                os.makedirs('./map/getOn')
            self.map[0].save('./map/getOn/{}.html'.format(self.ym))    
            
        #mt == 1 하차
        elif self.mt == '2':
            if os.path.exists('./map/getOff'):
                if os.path.exists('./map/getOff/{}.html'.format(self.ym)):
                    os.remove('./map/getOff/{}.html'.format(self.ym))
            else:
                os.makedirs('./map/getOff')
            self.map[1].save('./map/getOff/{}.html'.format(self.ym))    
            
        #mt == 2 유동인구(승하차)
        elif self.mt == '3':
            if os.path.exists('./map/floatingPop'):
                if os.path.exists('./map/floatingPop/{}.html'.format(self.ym)):
                    os.remove('./map/floatingPop/{}.html'.format(self.ym))
            else:
                os.makedirs('./map/floatingPop')
            self.map[2].save('./map/floatingPop/{}.html'.format(self.ym))                
            
def setMap(dataF,dataYM,mt):
    globals()['map{}_{}'.format(dataYM,mt)] = mapping(dataF,dataYM,mt)
    
def openMap(dataYear,selType):
    try:
        if selType == '1':
            if os.path.exists('./map/getOn/{}.html'.format(dataYear)):
                BASE_DIR = os.path.dirname(os.path.abspath('map/getOn/{}.html'.format(dataYear)))
                webbrowser.open('file:' + os.path.join(BASE_DIR,'{}.html'.format(dataYear)))
            else:
                raise Exception(dataYear + ' file is not exist in selectType : 1')

        elif selType == '2':
            if os.path.exists('./map/getOff/{}.html'.format(dataYear)):
                BASE_DIR = os.path.dirname(os.path.abspath('map/getOff/{}.html'.format(dataYear)))                
                webbrowser.open('file:' + os.path.join(BASE_DIR,'{}.html'.format(dataYear)))
            else:
                raise Exception(dataYear + ' file is not exist in selectType : 2')
                
        elif selType == '3':
            if os.path.exists('./map/floatingPop/{}.html'.format(dataYear)):
                BASE_DIR = os.path.dirname(os.path.abspath('map/floatingPop/{}.html'.format(dataYear)))                
                webbrowser.open('file:' + os.path.join(BASE_DIR,'{}.html'.format(dataYear)))
            else:
                raise Exception(dataYear + ' file is not exist in selectType : 3')

        else:
            raise Exception('selectType : ' + selectType + ' is not corrected')

    except Exception as e:
        print(e)
        return False


json 처리 전 지역 수 :  250
json 처리 후 지역 수 :  25


<h1>클라이언트 딴에서 실행</h1>

In [50]:
#DATA = server와 연동해서 server의 DATA값을 가져와야 함
if __name__ == '__main__':
    global DATA
    while True:
        dataYM,selT = map(str,input('Input dataYear (ex:202010) & Select Type (1:승차 / 2:하차 / 3:유동인구)\n').split(' '))
        try:
            for datayear in DATA.keys():
                if not len(dataYM) == 6 or not dataYM[:4] in datayear:
                    print("{} isn't correct year".format(dataYM))
                    break
                else:
                    setMap(DATA[datayear],str(dataYM),selT)
            if not openMap(dataYM,selT) == False:
                break
        except:
            break

    
    


Input dataYear (ex:202010) & Select Type (1:승차 / 2:하차 / 3:유동인구)
202012 3
202012 isn't correct year
202012 file is not exist in selectType : 3
Input dataYear (ex:202010) & Select Type (1:승차 / 2:하차 / 3:유동인구)
201910 3
201910 isn't correct year
